# Typical use cases

## Problem

In this notebook, we provide examples of different solver configurations a for sample Knapsack Problem with three items. The goal is to put the selected items in the knapsack to achieve the maximal cost with total weight not exceeding the `max_weight`.  The `items_weights` and `items_values` fields specify the weight and cost of each item, respectively.

```yaml
problem:
  type: knapsack
  max_weight: 2
  items_weights: [1, 1, 1]
  items_values: [2, 2, 1]
```

## Configuration for QAOA
The configuration below shows how to create a QAOA instance with 5 layers and a local gradient descent optimizer (`QmlGradientDescent`) --- by default the 'Adam' optimizer.

`angles` indicate variational parameters searched by specified optimizer;
`hyper args` refer to the initial weights in the objective function of the Knapsack Problem..

In [12]:
qaqa_config_yaml = """
problem:
  type: knapsack
  max_weight: 2
  items_weights: [1, 1, 1]
  items_values: [2, 2, 1]
solver:
  type: vqa
  pqc:
    type: qaoa
    layers: 5
  optimizer:
    type: qml
  params_inits:
    angles: [[0.5, 0.5, 0.5, 0.5, 0.5], [1, 1, 1, 1, 1]]
    hyper_args: [1, 2.5, 2.5]
"""

In [13]:
import yaml
from QHyper.solvers import solver_from_config

config = yaml.safe_load(qaqa_config_yaml)

solver = solver_from_config(config)

In [14]:
results = solver.solve()
results.probabilities

rec.array([(0, 0, 0, 0, 0, 0.01925662), (0, 0, 0, 0, 1, 0.00499353),
           (0, 0, 0, 1, 0, 0.06602495), (0, 0, 0, 1, 1, 0.00135034),
           (0, 0, 1, 0, 0, 0.00929764), (0, 0, 1, 0, 1, 0.00077379),
           (0, 0, 1, 1, 0, 0.0150905 ), (0, 0, 1, 1, 1, 0.00105546),
           (0, 1, 0, 0, 0, 0.01907666), (0, 1, 0, 0, 1, 0.0119462 ),
           (0, 1, 0, 1, 0, 0.01663426), (0, 1, 0, 1, 1, 0.0009503 ),
           (0, 1, 1, 0, 0, 0.0058822 ), (0, 1, 1, 0, 1, 0.11219791),
           (0, 1, 1, 1, 0, 0.01533116), (0, 1, 1, 1, 1, 0.00056568),
           (1, 0, 0, 0, 0, 0.01907666), (1, 0, 0, 0, 1, 0.0119462 ),
           (1, 0, 0, 1, 0, 0.01663426), (1, 0, 0, 1, 1, 0.0009503 ),
           (1, 0, 1, 0, 0, 0.0058822 ), (1, 0, 1, 0, 1, 0.11219791),
           (1, 0, 1, 1, 0, 0.01533116), (1, 0, 1, 1, 1, 0.00056568),
           (1, 1, 0, 0, 0, 0.00744606), (1, 1, 0, 0, 1, 0.07094647),
           (1, 1, 0, 1, 0, 0.01561782), (1, 1, 0, 1, 1, 0.0122494 ),
           (1, 1, 1, 0, 0, 0.00071

## Configuration for D-Wave Advantage

Configuration for a grid search hyperoptimizer. The objective function penalties (`hyper_args`) are searched within specified `bounds` using provided `steps`. The objective function is solved with a quantum annealing device --- D-Wave Advantage.

In [7]:
advantage_config_yaml = """
problem:
  type: knapsack
  max_weight: 2
  items_weights: [1, 1, 1]
  items_values: [2, 2, 1]
solver:
  type: advantage
  num_reads: 100
  hyper_optimizer:
    type: grid
    steps: [1, 1, 1]
    bounds: [[1, 3], [1, 3], [1, 3]]
  params_inits:
    weights: [1, 1, 1]
"""

In [8]:
import yaml
from QHyper.solvers import solver_from_config

config = yaml.safe_load(advantage_config_yaml)

solver = solver_from_config(config)

In [9]:
results = solver.solve()
results.probabilities

rec.array([(1, 1, 0, 0, 1, 0.38, -4.), (0, 1, 1, 0, 1, 0.09, -3.),
           (1, 0, 1, 0, 1, 0.16, -3.), (1, 1, 1, 1, 1, 0.09, -3.),
           (1, 1, 1, 0, 1, 0.16, -3.), (0, 1, 0, 1, 0, 0.04, -2.),
           (1, 0, 0, 1, 0, 0.03, -2.), (1, 1, 0, 1, 0, 0.03, -2.),
           (1, 0, 1, 1, 0, 0.01, -1.), (0, 1, 1, 1, 0, 0.01, -1.)],
          dtype=[('x0', '<i4'), ('x1', '<i4'), ('x2', '<i4'), ('x3', '<i4'), ('x4', '<i4'), ('probability', '<f8'), ('energy', '<f8')])

## Advanced configuration

QHyper configuration of the QAOA variant (WF-QAOA) with 5 `layers` and  the local gradient descent Adam `optimizer` (qml). `angles` indicate  initial variational parameters optimized by the method. `hyper_args` refer to the initial objective function penalties searched within `hyper_optimizer` `bounds` by the `CEM` method. `processes`, `samples_per_epoch`, and `epochs` are parameters specific to the `CEM` method.

.. warning::
    The `CEM` method is computationally expensive and may require a significant amount of time to complete (~15 min). 

In [6]:
advance_config_yaml = """
problem:
  type: knapsack
  max_weight: 2
  items_weights: [1, 1, 1]
  items_values: [2, 2, 1]
solver:
  type: vqa
  pqc:
    type: wfqaoa
    layers: 5
    backend: default.qubit
  optimizer:
    type: qml
    optimizer: adam
    steps: 50
    stepsize: 0.01
  hyper_optimizer:
    type: cem
    processes: 4
    samples_per_epoch: 200
    epochs: 5
    bounds: [[1, 10], [1, 10], [1, 10]]
    disable_tqdm: False
  params_inits:
    angles: [[0.5, 0.5, 0.5, 0.5, 0.5], [1, 1, 1, 1, 1]]
    hyper_args: [1, 2.5, 2.5]
"""

In [6]:
import yaml
from QHyper.solvers import solver_from_config

config = yaml.safe_load(advance_config_yaml)

solver = solver_from_config(config)

In [7]:
results = solver.solve()
results.probabilities

100%|██████████| 200/200 [02:35<00:00,  1.28it/s]


rec.array([(0, 0, 0, 0, 0, 0.00069401), (0, 0, 0, 0, 1, 0.01476763),
           (0, 0, 0, 1, 0, 0.05957409), (0, 0, 0, 1, 1, 0.03294517),
           (0, 0, 1, 0, 0, 0.03042684), (0, 0, 1, 0, 1, 0.00883227),
           (0, 0, 1, 1, 0, 0.00654667), (0, 0, 1, 1, 1, 0.04354659),
           (0, 1, 0, 0, 0, 0.01130641), (0, 1, 0, 0, 1, 0.00159494),
           (0, 1, 0, 1, 0, 0.01265646), (0, 1, 0, 1, 1, 0.03360276),
           (0, 1, 1, 0, 0, 0.01778942), (0, 1, 1, 0, 1, 0.08637871),
           (0, 1, 1, 1, 0, 0.06138423), (0, 1, 1, 1, 1, 0.03077721),
           (1, 0, 0, 0, 0, 0.01130641), (1, 0, 0, 0, 1, 0.00159494),
           (1, 0, 0, 1, 0, 0.01265646), (1, 0, 0, 1, 1, 0.03360276),
           (1, 0, 1, 0, 0, 0.01778942), (1, 0, 1, 0, 1, 0.08637871),
           (1, 0, 1, 1, 0, 0.06138423), (1, 0, 1, 1, 1, 0.03077721),
           (1, 1, 0, 0, 0, 0.04008062), (1, 1, 0, 0, 1, 0.15231274),
           (1, 1, 0, 1, 0, 0.01425434), (1, 1, 0, 1, 1, 0.00320176),
           (1, 1, 1, 0, 0, 0.01442

## Results evaluation

After obtaining the results, we evaluate the solution by calculating the total cost and weight of the items in the knapsack.

In [4]:
from QHyper.util import (
    weighted_avg_evaluation, sort_solver_results, add_evaluation_to_results)

problem = solver.problem

# Evaluate results with weighted average evaluation
print("Evaluation:")
print(weighted_avg_evaluation(
    results.probabilities, problem.get_score,
    penalty=0, limit_results=10, normalize=True
))
print("Sorted results:")
sorted_results = sort_solver_results(
    results.probabilities, limit_results=5)
print(sorted_results)

# Add evaluation to results
results_with_evaluation = add_evaluation_to_results(
    sorted_results, problem.get_score, penalty=0)

for rec in results_with_evaluation:
    print(f"Result: {rec}, "
          f"Prob: {rec['probability']:.5}, "
          f"Evaluation: {rec['evaluation']:.5}")

Evaluation:
-1.669217721264391
Sorted results:
[(1, 1, 0, 0, 1, 0.14605589) (1, 0, 1, 0, 1, 0.09231208)
 (0, 1, 1, 0, 1, 0.09231208) (1, 0, 1, 1, 0, 0.06831021)
 (0, 1, 1, 1, 0, 0.06831021)]
Result: (1, 1, 0, 0, 1, 0.14605589, -4.), Prob: 0.14606, Evaluation: -4.0
Result: (1, 0, 1, 0, 1, 0.09231208, -3.), Prob: 0.092312, Evaluation: -3.0
Result: (0, 1, 1, 0, 1, 0.09231208, -3.), Prob: 0.092312, Evaluation: -3.0
Result: (1, 0, 1, 1, 0, 0.06831021, 0.), Prob: 0.06831, Evaluation: 0.0
Result: (0, 1, 1, 1, 0, 0.06831021, 0.), Prob: 0.06831, Evaluation: 0.0
